# <div style="text-align: right;font-family:Times New Roman">Приложение Б-2</div>
# <div style="text-align: center;font-family:Times New Roman">Предобработка ЦМР и карт ландшафтного покрова</div>

# Preprocessing of DEMs and Landcover data
# Предобработка ЦМР и карт ландшафтного покрова
---
In this notebook DEMs and Landcover data is being merged into single mosaic with the same resolution and shape as Sentinel-2 data. This algorythm can be implemented to any kind of raster data.

В данном блоконте ЦМР и карты ландшафтного покрова объединяются в единую мозаику с тем же разрешением и размерами, что и данные Sentinel-2. Этот алгоритм может быть применен к любым растровым данным.

### Importing dependencies
### Импорт необходимых библиотек

In [3]:
import os
import random
import re
import numpy as np
from glob import glob
import geopandas as gpd
from osgeo import gdal
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.merge
import rasterio.fill
from rasterio.io import MemoryFile
import fiona
import shapely

### Opening reference Sentinel-2 file
### Открытие файла Sentinel-2

In [6]:
sentinel = rio.open(r'F:\Work\CorrProizv\B1.tif')

### Opening data for preprocessing
### Открытие данных для предобработки

In [8]:
path = r"E:\Work\CorrProizv\WC\ESA_WORLDCOVER_10M_2020_V100\MAP\\"
dems = glob(path+"*\\*_Map.tif")
dems

['E:\\Work\\carbon\\altai\\ESA_WORLDCOVER_10M_2020_V100\\MAP\\ESA_WorldCover_10m_2020_v100_N54E084_Map\\ESA_WorldCover_10m_2020_v100_N54E084_Map.tif',
 'E:\\Work\\carbon\\altai\\ESA_WORLDCOVER_10M_2020_V100\\MAP\\ESA_WorldCover_10m_2020_v100_N54E081_Map\\ESA_WorldCover_10m_2020_v100_N54E081_Map.tif',
 'E:\\Work\\carbon\\altai\\ESA_WORLDCOVER_10M_2020_V100\\MAP\\ESA_WorldCover_10m_2020_v100_N51E084_Map\\ESA_WorldCover_10m_2020_v100_N51E084_Map.tif']

In [9]:
files = []
for dem in dems:
    pathfile = rio.open(dem)
    files.append(pathfile)
files

[<open DatasetReader name='E:/Work/carbon/altai/ESA_WORLDCOVER_10M_2020_V100/MAP/ESA_WorldCover_10m_2020_v100_N54E084_Map/ESA_WorldCover_10m_2020_v100_N54E084_Map.tif' mode='r'>,
 <open DatasetReader name='E:/Work/carbon/altai/ESA_WORLDCOVER_10M_2020_V100/MAP/ESA_WorldCover_10m_2020_v100_N54E081_Map/ESA_WorldCover_10m_2020_v100_N54E081_Map.tif' mode='r'>,
 <open DatasetReader name='E:/Work/carbon/altai/ESA_WORLDCOVER_10M_2020_V100/MAP/ESA_WorldCover_10m_2020_v100_N51E084_Map/ESA_WorldCover_10m_2020_v100_N51E084_Map.tif' mode='r'>]

### Merging
### Объединение

In [10]:
dem, dem_trans = rio.merge.merge(files, method = 'last')

### Reprojection
### Перепроецирование

In [11]:
with MemoryFile() as memfile:
    with memfile.open(
        driver='GTiff',
        height=dem.shape[1],
        width=dem.shape[2],
        count=1,
        dtype=dem.dtype,
        compress = 'lzw',
        crs='EPSG:4326',
        transform=dem_trans,
        BIGTIFF='YES',
        nodata = 0
    ) as temp:
        temp.write(dem)
        dsttransform, width, height = calculate_default_transform(
            'EPSG:4326', sentinel.crs, temp.meta['width'], temp.meta['height'], *temp.bounds)
        dem1 = np.zeros((height, width), dem.dtype)
        reproject(
            source=dem,
            destination=dem1,
            src_transform=dem_trans,
            src_crs='EPSG:4326',
            src_nodata = 0,
            dst_transform=dsttransform,
            dst_crs=sentinel.crs,
            dst_nodata = 0,
            num_threads = 4,
            resampling=Resampling.nearest)
        dem = dem1

### Clipping
### Обрезка

In [12]:
clipper = "E:\\Work\\CorrProizv\\clipper.gpkg"
with fiona.open(clipper, "r") as shapefile:
    shape = [feature["geometry"] for feature in shapefile]
with MemoryFile() as memfile:
    with memfile.open(
        driver='GTiff',
        height=dem.shape[0],
        width=dem.shape[1],
        count=1,
        dtype=dem.dtype,
        compress = 'lzw',
        crs=sentinel.crs,
        transform=dsttransform,
        BIGTIFF='YES',
        nodata = 0
    ) as temp:
        temp.write(dem,1)
        dem, dem_trans = rio.mask.mask(temp, shape,crop=True, filled=True)

### Resampling to the resolution of Sentinel-2 mosaic
### Приведение к разрешению как у мозаики Sentinel-2

In [13]:
dem1 = np.zeros((sentinel.shape), dem.dtype)
reproject(
    source = dem,
    destination = dem1,
    src_transform = dem_trans,
    src_crs=sentinel.crs,
    src_nodata = 0,
    dst_transform = sentinel.transform,
    dst_resolution = sentinel.res,
    dst_crs=sentinel.crs,
    dst_nodata = 0,
    num_threads = 4,
    resampling=Resampling.nearest)
dem = dem1

### Saving
### Сохранение

In [15]:
with rio.open(
    r"E:\Work\CorrProizv\Landcover.tif",
    'w',
    driver='GTiff',
    height=dem.shape[0],
    width=dem.shape[1],
    count=1,
    dtype=dem.dtype,
    compress = 'lzw',
    crs=files[1].crs,
    #crs=sentinel.crs,
    transform=dem_trans,
    #transform=sentinel.transform,
    BIGTIFF='YES',
    nodata = 0
) as outfile:
    outfile.write(dem)

#### Mosaic creation tools from this notebook are now avaliable in a Python library named [Remote Sensing Processor](https://github.com/simonreise/remote-sensing-processor)
#### Инструменты создания мозаик, использованные в этом блокноте доступны в библиотеке Python [Remote Sensing Processor](https://github.com/simonreise/remote-sensing-processor)